In [1]:
import numpy as np
from math import cos, sin, pi

# From Ryan Keating via mc-capolei
# https://raw.githubusercontent.com/mc-capolei/python-Universal-robot-kinematics/master/universal_robot_kinematics.py

# https://www.universal-robots.com/articles/ur/application-installation/dh-parameters-for-calculations-of-kinematics-and-dynamics/
# ****** Coefficients ******


# UR 5e

d1 = 0.1625
a2 = -0.425
a3 = -0.3922
d4 = 0.1333
d5 = 0.0997
d6 = 0.0996

d = np.array([d1, 0, 0, d4, d5, d6])
a = np.array([0, a2, a3, 0, 0, 0])
alph = np.array([pi / 2, 0, 0, pi / 2, -pi / 2, 0])

In [2]:
def AH(n, th):

    T_a = np.array(np.identity(4), copy=False)
    T_a[0, 3] = a[n - 1]
    T_d = np.array(np.identity(4), copy=False)
    T_d[2, 3] = d[n - 1]

    Rzt = np.array(
        [
            [cos(th[n - 1]), -sin(th[n - 1]), 0, 0],
            [sin(th[n - 1]), cos(th[n - 1]), 0, 0],
            [0, 0, 1, 0],
            [0, 0, 0, 1],
        ],
        copy=False,
    )

    Rxa = np.array(
        [
            [1, 0, 0, 0],
            [0, cos(alph[n - 1]), -sin(alph[n - 1]), 0],
            [0, sin(alph[n - 1]), cos(alph[n - 1]), 0],
            [0, 0, 0, 1],
        ],
        copy=False,
    )

    A_i = T_d @ Rzt @ T_a @ Rxa

    return A_i

In [3]:
from spatialmath import SE3, Twist3
from UR5e import HTrans

def get_joint_twists():
    # everything in the space frame aka base frame
    joint_twists = []
    
    # first joint
    axis = np.array([0, 0, 1]) # rotates around z, right hand rule
    point = np.array([0, 0, 0]) # a point on the axis of rotation
    twist = Twist3.UnitRevolute(axis, point)
    joint_twists.append(twist)
    
    # second joint
    axis = np.array([0, -1, 0])
    point = np.array([0, 0, d1])
    twist = Twist3.UnitRevolute(axis, point)
    joint_twists.append(twist)
    
    # third joint
    axis = np.array([0, -1, 0])
    point = np.array([a2, 0, d1])
    twist = Twist3.UnitRevolute(axis, point)
    joint_twists.append(twist)
    
    # fourth joint
    axis = np.array([0, -1, 0])
    point = np.array([a2 + a3, -d4, d1])
    twist = Twist3.UnitRevolute(axis, point)
    joint_twists.append(twist)
    
    # fifth joint
    axis = np.array([0, 0, -1])
    point = np.array([a2 + a3, -d4, d1 - d5])
    twist = Twist3.UnitRevolute(axis, point)
    joint_twists.append(twist)
    
    # sixth joint
    axis = np.array([0, -1, 0])
    point = np.array([a2 + a3, -d4 - d6, d1 - d5])
    twist = Twist3.UnitRevolute(axis, point)
    joint_twists.append(twist)
    
    return joint_twists

zero_config_fk = HTrans([0]*6, True)[-1]
zero_config_fk = SE3(zero_config_fk)    

def get_fk_from_twists(joint_angles):
    joint_twists = get_joint_twists()
    relative_transforms = []
    for idx, joint_twist in enumerate(joint_twists):
        angle = joint_angles[idx]
        transform = SE3(joint_twist.exp(angle))
        relative_transforms.append(transform)
        
    fk = zero_config_fk
    for transform in relative_transforms[::-1]:  # apply in reverse order
        fk = transform * fk
    return fk

def get_ur5e_jacobian_from_twists(angles, frame=None):
    if frame is None:
        frame = "body"
    joint_twists = get_joint_twists()
    relative_transforms = []
    for idx, joint_twist in enumerate(joint_twists):
        angle = angles[idx]
        relative_transforms.append(SE3(joint_twist.exp(angle)))
    jacobian = np.zeros([6, 6])
    twist_transform = SE3(np.eye(4))
    for idx in range(6):
        if idx > 0:
            twist_transform = twist_transform @ relative_transforms[idx-1]
        jacobian[:, idx] = twist_transform.Ad() @ joint_twists[idx].A  
    
    if frame == "space":
        return jacobian
    elif frame == "body":
        fk = zero_config_fk
        for transform in relative_transforms[::-1]:  # apply in reverse order
            fk = transform * fk
        return fk.inv().Ad() @ jacobian
    else:
        raise Exception(f"frame: {frame} not in (space, body)")

def get_body_twist_from_transform(desired_transform, current_transform):
    """
    Even though both desired_transform and current_transform are in space frame,
    this returns a twist in the body frame.
    """
    transform_from_desired = SE3(current_transform).inv().A @ desired_transform
    twist = SE3(transform_from_desired).log(twist=True)
    return twist

def get_body_twist(current, desired):
    # swap arguments to match Sam's code
    body_twist = get_body_twist_from_transform(desired, current)
    return body_twist

def get_space_twist(current, desired):
    body_twist = get_body_twist(current, desired)
    space_twist = SE3(current).Ad() @ body_twist
    return space_twist

def space_jacobian(joint_angles):
    return get_ur5e_jacobian_from_twists(joint_angles, frame="space")
    
def body_jacobian(joint_angles):
    return get_ur5e_jacobian_from_twists(joint_angles, frame="body")
    


In [4]:
# https://github.com/PaperXLV/DifferentialInverseKinematics
from Examples import task_one

In [5]:
# sam_output = task_one(trials=[35], modes=["DLS"])

In [6]:
# space_output = task_one(trials=[35], modes=["DLS"], jacobian_fn=space_jacobian, desired_twist_fn=get_space_twist)

In [7]:
# body_output = task_one(trials=[35], modes=["DLS"], jacobian_fn=body_jacobian, desired_twist_fn=get_body_twist)


In [8]:
# print(space_output[0][:5])
# positions = space_output[0]

In [9]:
'''
from Webots import *
# running space_output
positions = sam_output[0]
# To run the manipulator
i = 0
while robot.step(timestep) != -1:
    position = positions[i]
    [motor.setPosition(angle) for motor, angle in zip(joint_motors, position)]

    i += 1
    if i == len(positions):
        break
'''

'\nfrom Webots import *\n# running space_output\npositions = sam_output[0]\n# To run the manipulator\ni = 0\nwhile robot.step(timestep) != -1:\n    position = positions[i]\n    [motor.setPosition(angle) for motor, angle in zip(joint_motors, position)]\n\n    i += 1\n    if i == len(positions):\n        break\n'

In [10]:
# from ProjectResults import create_error_plot, create_ellipse_plot, waypoints

In [11]:
# Cosine similarity plot for space output actual and desired twist
# create_error_plot(np.array(space_output[3][0]), np.array(space_output[4][0]))

In [12]:
# Ellipse plot for space output positions
# create_ellipse_plot(np.array(space_output[2][0]), waypoints)

In [13]:
from spatialmath.base import trnorm

def sam_epsilon(current_pose, desired_pose):
    return np.linalg.norm(current_pose - desired_pose)

def space_epsilon(current_pose, desired_pose):
    twist = get_space_twist(current_pose, desired_pose)
    return np.linalg.norm(twist)

# Jacobian for the UR5e
def Jacobian(joint_angles):
    transforms = HTrans(np.array([joint_angle for joint_angle in joint_angles]), True)
    end_pos = transforms[-1][0:3, 3]
    transforms = [np.identity(4)] + transforms[:-1]
    
    j = np.zeros((len(transforms), len(transforms)))
    for i, T in enumerate(transforms):
        # linear component is the cross product of the axis of rotation (z axis) and the vector from joint to end effector
        z_axis = T[0:3, 2]
        joint_pos = T[0:3, 3]
        joint_to_end = np.cross(z_axis, end_pos - joint_pos)
        j[0:3, i] = joint_to_end
        # rotational component is the axis of rotation (z axis)
        j[3:6, i] = z_axis

    return j

def sam_get_space_twist(current, desired):
    # note, this left-multiplies rather than right multiplies
    # Space twist from current and desired transform
    # trnorm fixed errors with bad (non-orthonormal) rotation matrice
    s = SE3(trnorm(desired @ SE3(current).inv().A)).log(twist = True)
    # Above will fail when the rotation matrix is the identity. Resolve manually. 
    if np.isnan(s).any():
        s[0:3] = desired[0:3, 3] - current[0:3, 3]
        s[3:6] = 0
    return s



epsilon_fns = [sam_epsilon, space_epsilon]
jacobian_fns = [Jacobian, space_jacobian]
twist_fns = [sam_get_space_twist, get_space_twist]

outputs = []
for epsilon_fn in epsilon_fns:
  for jacobian_fn in jacobian_fns:
    for twist_fn in twist_fns:
      output = task_one(
        trials=[30], 
        epsilon_fn=epsilon_fn,
        jacobian_fn=jacobian_fn, 
        desired_twist_fn=twist_fn,
        modes=["DLS"])
      
      positions, iterations, end_effector_positions, desired_twists, actual_twists = output
      # outputs += [iterations]
      outputs += [positions]
# Appears to be no performance difference between the methods - entirely interchangable


In [17]:
print(len(outputs))
outputs = [position for output in outputs for position in output]
print(len(outputs))

8
1400


In [18]:

from Webots import *
# running space_output
positions = outputs
# To run the manipulator
i = 0
while robot.step(timestep) != -1:
    position = positions[i]
    # print(position)
    [motor.setPosition(angle) for motor, angle in zip(joint_motors, position)]

    i += 1
    if i == len(positions):
        break
